In [1]:
import PIL
from torchvision import transforms

def get_testimg():
    test_img = PIL.Image.open('test pictures\\Amnesty POI-6-3-1.png')
    return transforms.ToTensor()(test_img)

In [2]:
import torch
import os
from models.SR3Builder import SR3Builder
from utils.model_utils import load_model

sr3_models = os.listdir('saved models\\SR3+')
device = torch.device(0)
model_builder = SR3Builder()
model = model_builder.set_sr3plus().build()
model.to(device)
sr_images = []
for sr3_model in sr3_models:
    torch.cuda.empty_cache()
    model = load_model(model, sr3_model, 'saved models\\SR3+\\')
    img = get_testimg().unsqueeze(0).to(device)
    sr = model.sample(img)
    torch.cuda.empty_cache()
    sr_images.append(sr.squeeze().to('cpu').detach())

The model weights have been loaded from 'saved models\SR3+\SR3+ Standart DA GA 2.pt'


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 11.99 GiB of which 0 bytes is free. Of the allocated memory 26.01 GiB is allocated by PyTorch, and 20.54 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
from utils.tensor_utils import tensor2img

for idx, tensor in enumerate(sr_images):
    img_np = tensor2img(tensor)
    img = PIL.Image.fromarray(img_np)
    img.save(f"test pictures\\SR3\\{sr3_models[idx]}.png")